In [125]:
import pandas as pd

In [126]:
# Here we are initializing the dictionary
# When this method is called a "count" variable is passed
# This becomes the key to the trials, and an empty dictionary
# is filled in for the corresponding value
def make_dict(count):
    
   # print("MAKING DICT")
    main_dict[count] = {
        'STIM1' : None,
        'STIM2' : None,
        'POSITION1' : None,
        'POSITION2' : None,
        'KEYPRESS'  : None,
        'TIME'      : None,
        'RESPONSE'      : None
    }

            

In [127]:



def parse_row(img):



# this counter is used to count the number of trials we have
# and it is used for our dictionary key
    count=0

# we start a loop iterating through the rows
# and parse out our relevant info 
    for row in img['img']:

#____________________________________________________________________________________ #   

    #check for the row w/ image (letters), and position
    # this conditional statement also iterates our "count" variable
    # and initializes a new dictionary key 
        if 'jpg' in row:
            #print(row)
            count=count+1
            ct = str(count)
            make_dict(ct)
        
        
            stim1 = row.split(" ")[1].split(".")[0]
            position1 = row.split(" ")[3].split("=")[1]
    #    print("Response1: ", response1)
     #   print("Position1: ", position1)
            stim2 = row.split(" ")[5].split(".")[0]
            position2 = row.split(" ")[7].split("=")[1]
         #   print("Response2: ", response2)
          #  print("Position2: ", position2)
            #print(count)
        
    # here we add our dictionary values 
            main_dict[ct]["STIM1"] = stim1
            main_dict[ct]["STIM2"] = stim2
            main_dict[ct]["POSITION1"] = position1
            main_dict[ct]["POSITION2"] = position2        

#____________________________________________________________________________________ #   
    # check for row with keypress and the time     
        if "time" in row:
            #print(row)
            keypress=row.split(" ")[1].split("=")[1]
    #    print(keypress)
            time=row.split(" ")[4]
        
            # here we add our dictionary values 
            main_dict[ct]["KEYPRESS"] = keypress
            main_dict[ct]["TIME"] = time 

            
#____________________________________________________________________________________ #   

     # check for row with a MISSED signal    
        if "Missed" in row: 
            keypress="Missed"
    # here we add our dictionary values 
            main_dict[ct]["KEYPRESS"] = keypress
            main_dict[ct]["TIME"] = "missed"
    #print(main_dict)


In [128]:
def get_data(file):
    df = pd.read_csv(file, engine='python', sep='\\t')
    df.columns=['onset', 'img', 'keypress']
    df=df.drop( columns=['keypress', 'onset'])
    # Here we grab a dataframe we call 'img', this contains the 
    # relavant column of our data we grabbed from the original df
    img = df[df['img'].str.contains("=|Missed")]
    return img 

In [129]:
def get_stim():
    for key in main_dict:
        keypress = main_dict[key]['KEYPRESS']
        pos1 = main_dict[key]['POSITION1']
        pos2 = main_dict[key]['POSITION2']
        stim1 = main_dict[key]['STIM1']
        stim2 = main_dict[key]['STIM2']
        if keypress != "Missed":
           # print("KEYPRESS: ", keypress)
            if pos1 == keypress:
                response = stim1
                #print("POS1: ", pos1)
                #print("RESP1: ", resp1)
                #print("STIM: ", stim)
                main_dict[key]['RESPONSE'] = response
            if pos2 == keypress:
                response = stim2
                #print("POS2: ", pos2)
                #print("RESP2: ", resp2)
                #print("STIM: ", stim)
                main_dict[key]['RESPONSE'] = response
        else:
            main_dict[key]['RESPONSE'] = "missed"
            
        
        

In [190]:
def create_output():
    global stat_df 
    global r_aCount # place holder for total count of response 'a'
    global r_bCount # " " ....total count of response 'b'
    global s_aCount # " " ....total count of stimulus 'a'
    global s_bCount # " " ....total count of stimulus 'b'
    stat_df = pd.DataFrame()
    output_df = pd.DataFrame()
    responses=[]
    stims = []
  
    
    r_aCount = 0 
    r_bCount = 0
    s_aCount = 0 
    s_bCount = 0
    
    
    #start a loop going over the trials, the loop currently is used to format
    #output format and get the statistics on the response and stimulus 
    for key in main_dict: 
        stim1 = main_dict[key]["STIM1"]
        stim2 = main_dict[key]["STIM2"]
        response = main_dict[key]["RESPONSE"]
        
        #this is adding relevant variables to our lists 
        stim = stim1+stim2
        stims.append(stim)
        responses.append(response)
        
        #the if statement is used to count our data-
        #a and b reponse and stimulus totals 
        if response == 'b':
            r_bCount = r_bCount + 1
        if response == 'a':
            r_aCount = r_aCount + 1
        if stim1 == "a" or stim2 == "a":
            s_aCount+=1
        if stim1 == "b" or stim2 == "b":
            s_bCount = s_bCount + 1
    
    a_posChoice = r_aCount/s_aCount * 100
    b_posChoice = r_bCount/s_bCount * 100 
    a_negChoice = 100 - a_posChoice
    b_negChoice = 100 - b_posChoice 
    a_posChoice = "%0.2f" % a_posChoice
    a_negChoice = "%0.2f" % a_negChoice
    b_posChoice = "%0.2f" % b_posChoice
    a_posChoice = "%0.2f" % a_posChoice
    
    
    output_df["STIM"] = stims
    output_df["RESPONSE"] = responses 
 
    stat_df["STIMULUS"] = [s_aCount, s_bCount]
    stat_df["RESPONSE"] = [r_aCount, r_bCount]
    stat_df["POSITIVE"] = [a_posChoice, b_posChoice]
    stat_df["NEGATIVE"] = [a_negChoice, b_negChoice ]
    
    print("STIMULUS COUNT A: %s \t COUNT B: %s"%(s_aCount,s_bCount))
    print("RESPONSE COUNT A: %s \t COUNT B: %s"%(r_aCount, r_bCount))

    #makes columns from our lists made above and makes a new dataframe 
       
    
    print(output_df.head())
    
    stat_df.rename(index={0:'a', 1:'b'}, inplace=True)
    print(stat_df.head())

In [191]:
def main():
    global main_dict
    main_dict={}
    file = '/Users/nikkibytes/Documents/RW_bevel/test_logs/beveltest_bevel01_2018-03-19-17_13_06_subdata.log'
    
    img_df = get_data(file)
    parse_row(img_df)
    get_stim()
    #print(main_dict)
    #print("output")
    create_output()
    
main()

STIMULUS COUNT A: 27 	 COUNT B: 21
RESPONSE COUNT A: 14 	 COUNT B: 11
  STIM RESPONSE
0   be   missed
1   fa        f
2   ea        a
3   be        e
4   ad        d
   STIMULUS  RESPONSE POSITIVE   NEGATIVE
a        27        14    51.85  48.148148
b        21        11   52.381  47.619048
